# A Basic Capacity Expansion Model

_**[Power Systems Optimization](https://github.com/east-winds/power-systems-optimization)**_

_by Jesse D. Jenkins and Michael R. Davidson (last updated: September 21, 2020)_

This notebook presents a basic electricity generation capacity expansion model: the task of minimizing the cost of fixed and variable costs across a fleet of generators to meet anticipated future electricity demand. 

We first present a basic optimization formulation of this optimal thermal generator capacity expansion problem, assuming a "greenfield" setting with no existing generating capacity. We also show how to create this model with Julia + JuMP.

Next, we discuss the impact of variable renewable energy sources (wind, solar) and present a slightly modified formulation that co-optimizes thermal and renewable capacities. 

Finally, we will note how this problem generalizes to a "brownfield" expansion problem including existing generators, with optimal retirement decisions included in the problem.

Note these capacity expansion problems will be basic/simplified formulations, ignoring inter-temporal operating constraints (e.g. ramp limits), energy storage, network constraints and geospatial tradeoffs, unit commitment decisions for thermal generators, and operating reserves. We will define a more complex model integrating these factors in a future notebook after familiarizing ourselves with operational models (economic dispatch, unit commitment, and optimal power flow).

## 1. Optimal thermal generator expansion problem

<img src="img/thermal.png" style="width: 450px; height: auto" align="left">

Assume a utility can build any of the following four thermal generation types: geothermal, coal, natural gas combined cycle gas turbine (CCGT), and natural gas combustion turbine (CT). The utility must decide on the mix of thermal generators to meet forecasted demand in a future planning year across each hour of the year. 

Assume for simplicity that there are no existing generators -- that is, this is a "greenfield" expansion problem where all generators are newly built. 

Finally, assume that any involuntary curtailment of demand caused by shortfalls in supply is penalized at the cost of non-served energy. (Note that by penalizing the opportunity cost of demand curtailment, this cost minimization problem is equivalent to a social welfare maximization problem, assuming inelastic demand). 

### Problem formulation

This yields the following **optimization problem**:

$\begin{align}
\min &\sum_{g \in G} \left(FixedCost_g \times CAP_g +  \sum_{h \in H} VarCost_g \times GEN_{g,h}\right)& \\
&\quad\quad+ \sum_{h \in H} NSECost \times NSE_h &\\[6pt]
&\text{s.t.}&\\[6pt]
& \sum_{g \in G} GEN_{g,h} + NSE_h = Demand_h & \forall \quad h \in H\\[6pt]
& GEN_{g,h} \leq CAP_g & \forall \quad g \in G \text{ & } h \in H\\[6pt]
& CAP_{g} \geq 0 & \forall \quad g \in G\\[6pt]
& GEN_{g,h} \geq 0 & \forall \quad g \in G \text{ & } h \in H\\
& NSE_{h} \geq 0 & \forall \quad h \in H
\end{align}$

And where:

$\begin{align}
&FixedCost_g = Capex_g \times CRF_g + FixedOM_g\\[6pt]
&CRF_g = \frac{WACC(1+WACC)^{Life}}{(1+WACC)^{Life}-1} \\[6pt]
&VarCost_g = VarOM_g + HeatRate_g \times FuelCost_g
\end{align}$

The **decision variables** in the above problem are:
- $CAP_{g}$, the capacity (in MW) built for each generation type, $g$; and
- $GEN_{g,h}$, the generation (in MWh) produced by each generator, $g$, in each hour, $h$; and
- $NSE_h$, the quantity of involuntarily curtailed demand in each hour (in MWh), $h$.

Note that if we ignore the discrete nature of large thermal power plant decisions (investment and power plant cycling decisions), we have continous decisions and *a linear programming (LP) problem* with linear objective function and linear constraints. (We'll incorporate discrete decisions in the notebook on Unit Commitment)

The **sets** are:
- $G$, the set of generators: \[geo, coal, CCGT, CT\];
- $H$, the set of hours in the year: \[1:8760\]

The **parameters** are:

- $Capex_g$, the capital expenditure to construct the power plant (overnight cost + cost of financing during construction) (\$/MW); 
- $CRF_g$,  the annual capital recovery factor or share of $Capex$ recovered in each year to pay off equity and debt (per unit);
- $WACC_g$,  the weighted average cost of capital to finance the generator, or the weighted cost of interest on debt and returns to equity investors (\%); 
- $Life_g$, the financial asset life of the asset (years); 
- $FixedOM_g$, the fixed operations and maintenance cost (\$/MW-year)
- $VarOM_g$, the variable operations and maintenance cost (\$/MWh);
- $HeatRate_g$, the rate of fuel consumed per unit of electricity produced (MMBtu/MWh);
- $FuelCost_g$, the cost of fuel (\$/MMBtu)
- $NSECost$, the opportunity cost or penalty incurred for involuntary non-served energy (\$/MWh); and
- $Demand_h$, is the demand in each hour

## Building the problem with Julia + JuMP

Now let's define this LP problem with the help of Julia + JuMP.

**1. Load packages**

First, let's load the Julia packages we'll need to work with here.

In [ ]:
# Uncomment this line if you need to install these packages:
# import Pkg; Pkg.add("JuMP"); Pkg.add("Clp"); Pkg.add("DataFrames"); Pkg.add("CSV"); Pkg.add("Statistics")
using JuMP, Clp
using Plots; plotly()
using DataFrames, CSV, Statistics
ENV["COLUMNS"]=120 # Set so all columns of DataFrames and Matrices are displayed

**1. Load parameters (data)**

Then we'll load our parameters, starting with the cost parameters for the candidate generators.

The following generator data are illustrative and approximate, but most are based on the National Renewable Energy Laboratory's [Annual Technology Baseline 2020](https://atb.nrel.gov/electricity/2020/), except for natural gas CT capex, which is based on Lazard's [Levelized Cost of Electricity Version 13.0](https://www.lazard.com/media/451086/lazards-levelized-cost-of-energy-version-130-vf.pdf). 

In [ ]:
generators = DataFrame(CSV.File("expansion_data/generators_for_expansion.csv"))

Note we've got wind and solar costs in here, but we'll get to these variable renewable resources in the next section.

Next we'll load the hourly demand data. This demand time series is the actual reported demand (from 2012) for San Diego Gas and Electric (via the [PowerGenome](https://github.com/gschivley/PowerGenome) data platform as reported to FERC Form 714).

In [ ]:
demand = DataFrame(CSV.File("expansion_data/demand_for_expansion.csv"))

Let's plot the demand to get a feel for what we're working with here...

In [ ]:
plot(demand.Demand, ylims=(0,5000), label="")
title!("Hourly Demand")
xaxis!("Hour")
yaxis!("MWh")

(Note that this time series starts at 0\:00-0\:59 [UTC](https://en.wikipedia.org/wiki/Coordinated_Universal_Time) time, or 5pm Pacific time)

You can see some of the seasonal trends here in San Diego electricity demand with some large peaks in demand in the summer months driven by air conditioning needs. This profile also includes the impact of about 600 MW of distributed solar PV installed behind the meter, which reduces the net load for the bulk generators we are planning here.

The chronological demand profile is a little hard to look at though, with all the daily variation, and since we're not considering intertemporal constraints (yet), let's resort the demand from highest to lowest hour of demand across the year, what is known as a **"load duration curve."**

In [ ]:
plot(sort(demand.Demand, rev=true), ylims=(0,5000), label="")
title!("Hourly Demand")
xaxis!("Hour")
yaxis!("MWh")

Examining the load duration curve, we can more easily see the wide range of variation in hourly demand across the year...

In [ ]:
print(string("Minimum demand: ",minimum(demand.Demand),"\n"))
print(string("Maximum demand: ",maximum(demand.Demand),"\n"))
print(string("Median demand: ",median(demand.Demand),"\n"))
print(string("Mean demand: ",round(mean(demand.Demand),digits=0),"\n"))

Finally, we will set the penalty for non-served energy or $NSECost$ to \$9,000/MWh

In [ ]:
NSECost = 9000

**3. Define sets**

Now let's define the sets we are going to use to index our variables:

- $G$, the set of generators: \[geo, coal, CCGT, CT\];
- $H$, the set of hours in the year: \[1:8760\]

In [ ]:
# The set of generators from the generators DataFrame
G = generators.G[1:(size(generators,1)-2)]  # note we truncate wind and solar for now
# The set of hours in the demand DataFrame
H = demand.Hour

**4. Define the model and specify solver**

In [ ]:
Expansion_Model = Model(Clp.Optimizer)

**5. Define variables**

Next, define decision variables: 

- $CAP_{g} \geq 0$, the capacity (in MW) built for each generation type, $g$; and
- $GEN_{g,h} \geq 0$, the generation (in MWh) produced by each generator, $g$, in each hour, $h$; and
- $NSE_h \geq 0$, the quantity of involuntarily curtailed demand in each hour, $h$.

In [ ]:
@variables(Expansion_Model, begin
        CAP[g in G] >=0          # Generating capacity built (MW)
        GEN[g in G, h in H] >= 0 # Generation in each hour (MWh)
        NSE[h in H] >= 0         # Non-served energy in each hour (MWh)
end)

**6. Define constraints**

Set up our constraints:

$\begin{align}
& \sum_{g \in G} GEN_{g,h} + NSE_h = Demand_h & \forall \quad h \in H\\[6pt]
& GEN_{g,h} \leq CAP_g & \forall \quad g \in G \text{ & } h \in H\\[6pt]
\end{align}$

In [ ]:
@constraints(Expansion_Model, begin
    cDemandBalance[h in H], sum(GEN[g,h] for g in G) + NSE[h] == demand.Demand[h]
    cCapacity[g in G, h in H], GEN[g,h] <= CAP[g]
end)

**7. Define objective function**

$\begin{align}
\min &\sum_{g \in G} \left(FixedCost_g \times CAP_g +  \sum_{h \in H} VarCost_g \times GEN_{g,h}\right)& \\
&\quad\quad+ \sum_{h \in H} NSECost \times NSE_h &\\[6pt]
\end{align}$

In [ ]:
# The output of this cell is a formula same as the one above, with all nested summations fully expanded,
# which is (MASSIVELY) LONG and makes my jupyter hang.
# To see the output, remove the semicolon ";" at the end of this code block; Make a backup if you want to
# do so, else it'll be very difficult to recover.
@objective(Expansion_Model, Min,
    sum(generators[generators.G.==g,:FixedCost][1]*CAP[g] + 
        sum(generators[generators.G.==g,:VarCost][1]*GEN[g,h] for h in H)
    for g in G) + 
    sum(NSECost*NSE[h] for h in H) 
);

**8. Solve the model and extract results**

In [ ]:
optimize!(Expansion_Model)

In [ ]:
# Record generation capacity and energy results
generation = zeros(size(G,1))
for i in 1:size(G,1) 
    generation[i] = sum(value.(GEN)[G[i],:].data) 
end
MWh_share = generation./sum(demand.Demand).*100
cap_share = value.(CAP).data./maximum(demand.Demand).*100
results = DataFrame(
    Resource = G, 
    MW = value.(CAP).data,
    Percent_MW = cap_share,
    GWh = generation/1000,
    Percent_GWh = MWh_share
)
# Calculate how much non-served energy there was and add to results
  # The maximum MW of non-served energy is the difference 
  # between peak demand and total installed generation capacity
NSE_MW = maximum(value.(NSE).data) 
  # The total MWh of non-served energy is the difference between
  # total demand and total generation
NSE_MWh = sum(value.(NSE).data)
  # Add or "push" a new row of data to the end of the results DataFrame
push!(results, ["NSE" NSE_MW NSE_MW/maximum(demand.Demand)*100 NSE_MWh/1000 NSE_MWh/sum(demand.Demand)*100])

You can see that given the cost assumptions provided, the least-cost capacity mix includes exclusively gas-fired generators: a mix of CCGTs which operate at higher utilization rates given their comparatively higher fixed costs and lower variable costs and CTs which run infrequently (to meet peaks in demand) given their lower fixed costs and higher fuel costs.

CCGTs make up 65% of capacity and CTs 32%, while CCGTs produce nearly 97% of annual generation and CTs provide only about 3%. A maximum of 3% of demand is curtailed in any hour (184 MW) and no more than 0.002% of demand across the year is unserved (99.998% reliability). 

What might happen if you changed the fuel costs? Or reduced the penalty for non-served energy? Questions to consider for Homework 2...

## 2. Co-optimizing thermal generators and variable renewables

<img src="img/renewables.png" style="width: 450px; height: auto" align="left">

Now let's add some wind and solar to the mix. 

We already loaded the cost data for wind and solar previously.

In [ ]:
generators

We will also need time series data containing the hourly *capacity factor* or maximum availability of wind and solar per unit of capacity deployed. This reflects the weather-dependent variation in potential wind and solar output across each hour of the year.

In [ ]:
variability = DataFrame(CSV.File("expansion_data/wind_solar_for_expansion.csv"))

Let's plot the first month of data (January), to get a feel for the patterns of variability...

In [ ]:
plot(variability.Wind[1:(24*31)], ylims=(0,1), label="Wind")
plot!(variability.Solar[1:(24*31)], ylims=(0,1), label="Solar")
title!("Capacity Factors for Wind & Solar - January")
xaxis!("Hour")
yaxis!("Capacity Factor")

And here's what July (the peak month for demand) looks like...

In [ ]:
july1 = 24*(31*3+30*2+28)
july31 = 24*(31*4+30*2+28)
plot(variability.Wind[july1:july31], ylims=(0,1), label="Wind")
plot!(variability.Solar[july1:july31], ylims=(0,1), label="Solar")
title!("Capacity Factors for Wind & Solar - July")
xaxis!("Hour")
yaxis!("Capacity Factor")

Now we have to slightly modify our problem formulation. For variable renewable resources $g \in RE$, where $RE$ is the subset $\{Wind, Solar\} \subset G$, the maximum possible generation is now time-dependent and equal to $CAP_g \times cf_{g,h}$ where $cf_{g,h}$ is the capacity factor for resource $g$ in hour $h$. That is, a 100 MW solar farm with an hourly capacity factor of 0.2 (20%) can produce no more than 20 MW in that hour. (We assume these are curtailable, utility-scale wind and solar PV plants that can produce less than this maximum amount of desired).

This yields:

$\begin{align}
\min &\sum_{g \in G} \left(FixedCost_g \times CAP_g +  \sum_{h \in H} VarCost_g \times GEN_{g,h}\right)& \\
&\quad\quad+ \sum_{h \in H} NSECost \times NSE_h &\\[6pt]
&\text{s.t.}&\\[6pt]
& \sum_{g \in G} GEN_{g,h} + NSE_h = Demand_h & \forall \quad h \in H\\[6pt]
& GEN_{g,h} \leq CAP_g & \forall \quad g \notin RE \text{ & } h \in H\\[6pt]
& GEN_{g,h} \leq CAP_g \times cf_{g,h} & \forall \quad g \in RE \text{ & } h \in H\\[6pt]
& CAP_{g} \geq 0 & \forall \quad h \in H\\[6pt]
& GEN_{g,h} \geq 0 & \forall \quad g \in G \text{ & } h \in H\\
& NSE_{h} \geq 0 & \forall \quad h \in H
\end{align}$

In Homework 2, you can code up this version of the generation expansion problem.

## 3. Brownfield expansion and retirement decisions

<img src="img/brownfield.png" style="width: 450px; height: auto" align="left">

The formulation above assumes no existing generating capacity -- a "greenfield" expansion problem.

But realistic planning problems will begin with today's installed capacity and then consider both what to build *and what to retire* to meet future demand at lowest cost. 

The capital expenditure to build existing generators is a *sunk cost*, so this should not factor into the economic decision making as to whether to continue operating a power plant. The relevant decisions have to do only with *avoidable costs*, in this case, ongoing fixed operations and maintenance costs (which we will consider inclusive of any capital expenditure to refurbish or replace aging equipment and continue operating). 

Let's define a formulation for this "brownfield" generation expansion problem with existing capacity retirement decisions and new build capacity expansion decisions.

First, define a new subset of generators $OLD \subset G$ that consists of all existing generators, and an existing capacity parameter $ExistingCap_g \forall g \in OLD$. 

For these resources, we don't have a capacity expansion decision ($CAP_g$), but we do have a retirement decision, $RET_g$, which indicates how much capacity is to be retired before the planning year. 

The $FixedCost_g$ for each existing generator consists only of the ongoing fixed operations and maintenance costs ($FixedOM_g$), with no annuitized investment cost component.

For convenience, let's also define a new subset $NEW \subset G$ that consists of all new build capacity options.

The new formulation is as follows:

$\begin{align}
\min &\sum_{g \in NEW} FixedCost_g \times CAP_g + \sum_{g \in OLD} FixedCost_g \times (ExistingCap_g - RET_g)& \\
&\quad\quad+ \sum_{g\in G}\sum_{h \in H} VarCost_g \times GEN_{g,h} + \sum_{h \in H} NSECost \times NSE_h &\\[6pt]
\end{align}$

$\begin{align}
&\text{s.t.}&\\[6pt]
&\text{Demand balance constraint:}&\\[6pt]
& \sum_{g \in G} GEN_{g,h} + NSE_h = Demand_h & \forall \quad h \in H\\[6pt]
&\text{Capacity constraint for new build thermal:}&\\[6pt]
& GEN_{g,h} \leq CAP_g & \forall \quad g \in NEW \text{ & } \notin RE \text{ & } h \in H\\[6pt]
&\text{Capacity constraint for existing thermal:}&\\[6pt]
& GEN_{g,h} \leq ExistingCap_g - RET_g & \forall \quad g \in OLD \text{ & } \notin RE \text{ & } h \in H\\[6pt]
&\text{Capacity constraint for new build RE:}&\\[6pt]
& GEN_{g,h} \leq CAP_g \times cf_g & \forall \quad g \in NEW \text{ & } \in RE \text{ & } h \in H\\[6pt]
&\text{Capacity constraint for existing RE:}&\\[6pt]
& GEN_{g,h} \leq (ExistingCap_g - RET_g) \times cf_g & \forall \quad g \in OLD \text{ & } \in RE \text{ & } h \in H\\[6pt]
&\text{Capacity variable only for new build:}&\\[6pt]
& CAP_{g} \geq 0 & \forall \quad g \in NEW\\[6pt]
&\text{Retirement variable only for existing:}&\\[6pt]
& RET_{g} \geq 0 & \forall \quad g \in OLD\\[6pt]
&\text{Generation and NSE variables:}&\\[6pt]
& GEN_{g,h} \geq 0 & \forall \quad g \in G \text{ & } h \in H\\
& NSE_{h} \geq 0 & \forall \quad h \in H
\end{align}$

You'll construct this model in Homework 2 as well.